In [3]:
from pathlib import Path
import json
from openai._client import OpenAI
from pprint import pprint

In [4]:
with open("../apikeys/apikeys.json") as f:
    api_keys = json.load(f)

### Completion

In [72]:
class Bot:
    def __init__(
        self, 
        init_prompt, 
        base_url='https://oneapi.xty.app/v1', 
        api_key='none',
        model="gpt-3.5-turbo", 
        temperature=0.7,
        max_tokens=16000,
    ):
        self.init_prompt = init_prompt
        self.client = OpenAI(base_url=base_url, api_key=api_key)
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.messages = [{"role": "system", "content": self.init_prompt}]

    @property
    def n_tokens(self):
        return sum([len(m['content']) for m in self.messages])

    def chat(self, content, print=True):
        try:
            self.drop_oldest_if_message_too_long(token_limit=self.max_tokens - len(content))
            response = self.client.chat.completions.create(
                model=self.model,
                messages=self.messages + [{"role": "user", "content": content}],
                temperature=self.temperature,
            )
        except Exception as e:
            return self.handle_output(f"Error: {e}", print=print)

        self.messages.append({"role": "user", "content": content})
        resp_message = response.choices[0].message.dict()
        resp_message.pop("function_call")
        resp_message.pop("tool_calls")
        self.messages.append(resp_message)
        return self.handle_output(resp_message["content"], print=print)
    
    def drop_oldest_if_message_too_long(self, token_limit):
        while self.n_tokens > token_limit:
            del self.messages[1]  # always delete the oldest one except for the init_prompt
    
    def handle_output(self, output_content, print=True):
        if not print:
            return output_content
        pprint(output_content)
        

In [73]:
init_prompt = """You are an expert in Javascript and meanwhile a JavaScript teacher that teaches students basics of Javascript. 
- You should provide explanations and examples for basic JavaScript syntax and concepts. 
- As a user interacts with you, they expect patient guidance and detailed explanations. 
- You should not only answer the user's specific questions but also provide additional related information to help the user expand their understanding of JavaScript. 
- When answering questions, you should use simple language and provide practical examples to illustrate the concepts. - You should aim to empower the user to learn beyond the immediate question and encourage them to explore similar ideas and usages in JavaScript.
"""

In [74]:
bot = Bot(init_prompt, model="gpt-3.5-turbo", api_key=api_keys["3.5"])

In [64]:
bot.chat("In a Three.js based project, I have two THREE.Group() in `scene.children`, say A and B, is there any way to create a toggle to select either displaying A or B exclusively?")

('Yes, you can create a toggle to display either group A or group B '
 'exclusively in a Three.js project. You can achieve this by adding and '
 "removing the desired group from the scene based on the user's selection.\n"
 '\n'
 "Here's a basic example of how you can achieve this:\n"
 '\n'
 '```javascript\n'
 '// Assuming scene is your Three.js scene and groupA, groupB are your '
 'THREE.Group objects\n'
 '\n'
 'let displayGroupA = true;\n'
 'let displayGroupB = true;\n'
 '\n'
 '// Function to toggle display of group A\n'
 'function toggleDisplayGroupA() {\n'
 '  displayGroupA = !displayGroupA; // Toggle the displayGroupA flag\n'
 '  if (displayGroupA) {\n'
 '    scene.add(groupA); // Add groupA to the scene\n'
 '  } else {\n'
 '    scene.remove(groupA); // Remove groupA from the scene\n'
 '  }\n'
 '}\n'
 '\n'
 '// Function to toggle display of group B\n'
 'function toggleDisplayGroupB() {\n'
 '  displayGroupB = !displayGroupB; // Toggle the displayGroupB flag\n'
 '  if (displayGroupB)

In [67]:
bot.chat("in jupyter notebook, how can I render a string containing \n and # as markdown format?")

('In a Jupyter notebook, you can render a string containing `#` and `##` as '
 "Markdown format using the IPython display module. Here's how you can achieve "
 'this:\n'
 '\n'
 '```javascript\n'
 '// Assuming you have a string containing Markdown content\n'
 'let markdownString = "## This is a heading\\nThis is a paragraph with some '
 '`code` and a [link](https://www.example.com)";\n'
 '\n'
 '// Render the Markdown string using IPython display\n'
 "const { display } = require('nodejs').default;\n"
 "display({ data: { 'text/markdown': markdownString }, metadata: {} });\n"
 '```\n'
 '\n'
 'In this example, we use the `display` function from the `nodejs` module to '
 'render the Markdown string. The `data` object contains the Markdown content, '
 'and the `text/markdown` type indicates that it should be rendered as '
 'Markdown.\n'
 '\n'
 'After running this code in a Jupyter notebook, you will see the Markdown '
 'content displayed as rendered Markdown, with the heading and paragraph '


In [69]:
a = [1, 2, 3, 4, 5]

In [70]:
del a[2]

In [71]:
a

[1, 2, 4, 5]

In [ ]:
def get_completion(prompt, message, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"], 

In [13]:
client = OpenAI(
    base_url='https://oneapi.xty.app/v1',
    api_key='none'
)

In [14]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[
        {"role": "system", "content": "You are an expert in Javascript and meanwhile a JavaScript teacher"},
        {"role": "user", "content": "Hello."},
    ]
)

In [15]:
msg = completion.choices[0]

In [21]:
msg.message.dict()

{'content': 'Hello! How can I assist you today?',
 'role': 'assistant',
 'function_call': None,
 'tool_calls': None}

In [53]:
a = msg.message.dict()